# Projeto 3 - Predição de varíavel quantitativa

----
## Introdução

Quando se trata de viagens, seja a passeio ou a trabalho, um cronograma pré-determinado é essencial para a organização da viagem. Entretanto, por mais que um cronograma seja montado pensando nas mais diversas variáveis, o atraso de voos nos aeroportos tende a pegar os viajantes de surpresa, sendo um dos mais frequentes fatores de alteração no cronograma dos viajantes. 

Pensando nisso, o objetivo deste projeto é criar um **modelo de predição de atrasos de voos domésticos nos EUA**, utilizando um dataset que contém informações de todos os voos domésticos ocorridos em 2023. Visando aumentar o desempenho do modelo, escolhemos prever o atraso dos voos apenas do **aeroporto de Atlanta**, de onde sairam a maior quantidade de voos em 2023. A variável principal (target) que será predita é o atraso total de voos em minutos..

O modelo será desenvolvido com o intuito de fornecer insights e previsões precisas sobre os atrasos de voos, auxiliando na tomada de decisões e no planejamento de viagens aéreas. Para isso, o projeto foi dividido em algumas etapas, sendo elas:
1) Importação das bibliotecas necessárias
2) Leitura do dataset
3) Limpeza dos dados
4) Análise exploratória dos dados
5) Divisão dos dados em treino e teste
6) Construção do modelo de predição
7) Avaliação do modelo
8) Conclusão
9) Referências


----
## Minerando os dados

Os dados utilizados neste projeto foram obtidos na plataforma Kaggle e contém informações sobre todos os voos domésticos ocorridos nos EUA em 2023. O dataset é composto por 6.743.404 linhas e 24 colunas, sendo que cada linha representa um voo e cada coluna representa uma variável.

Devido ao tamanho do dataset, foi necessário realizar um pré-processamento dos dados para que fosse possível realizar a análise exploratória e a construção do modelo de predição. O pré-processamento consistiu nas seguintes etapas:

1) Filtragem do dataset para conter apenas os voos que partiram de Atlanta;
2) Análise das variáveis e remoção das colunas que não seriam utilizadas no modelo, sendo elas:
    1) Descrição do avião (detalhes técnicos do modelo dos aviões);
    2) Nome das cidades de partida e chegada (código do aeroporto é mais relevante e o ponto de partida sempre será o aeroporto de Atlanta);
    3) Duração do voo (já existe uma coluna de classficação da duração do voo, a qual julgamos mais relevante);
3) Alterar a variável da data completa dos voos para apenas o mês dos voos, facilitando a generalização do modelo.  

Ao final do processo de mineração dos dados, a base de dados ficou composta por 332.935 linhas e 16 colunas.

----
## Características do dataset

Abaixo incluimos uma breve descrição das variáveis presentes no dataset:
| Atributos | Descrição | Possíveis Valores |
|------|------|------|
|   Month  | Mês em que os voos foram realizados | 1 a 12 |
|   Day_Of_Week  | Dia da semana em que os voos foram realizados | 1 a 7 |
|   Airline  | Nome da companhia aérea | Southwest Airlines Co., Delta Air Lines Inc, American Airlines Inc., United Air Lines Inc., Skywest Airlines Inc., Republic Airways, JetBlue Airways, Spirit Air Lines, Alaska Airlines Inc., American Eagle Airlines Inc., Endeavor Air, PSA Airlines, Frontier Airlines Inc., Allegiant Air, Hawaiian Airlines Inc.
|   DepTime_label  | Horário de partida dos voos | Morning, Afternon, Evening, Night |
|   Dep_Delay  | Atraso na partida dos voos | Quantidade de minutos | 
|   Dep_Delay_Tag  | Tag de atraso na partida dos voos | 0: Sem atraso ou atrasos menores que 5 minutos; 1: Atrasos maiores que 5 minutos |
|   Dep_Delay_Type  | Tipo de atraso na partida dos voos | Low <5min, Medium >15min, Hight >60min |
|   Arr_Airport  | Aeroporto de chegada dos voos | Código do aeroporto |
|   Arr_Delay  | Atraso na chegada dos voos | Quantidade de minutos |
|   Arr_Delay_Type  | Tipo de atraso na chegada dos voos | Low <5min, Medium >15min, Hight >60min |
|   Distance_type  | Tipo de distância do voo | Short Haul >1500Mi, Medium Haul <3000Mi, Long Haul <6000Mi |
|   Delay_Carrier  | Atraso causado pela companhia aérea | Quantidade de minutos |
|   Delay_Weather  | Atraso causado por condições climáticas | Quantidade de minutos |
|   Delay_NAS  | Atraso causado por problemas no sistema de controle de tráfego aéreo | Quantidade de minutos |
|   Delay_Security  | Atraso causado por problemas de segurança | Quantidade de minutos |
|   Delay_LastAircraft  | Atraso causado por problemas no avião | Quantidade de minutos |

----
## Bibliotecas e leitura dos dados

In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import *

In [2]:
# Leitura do dataset
dados = pd.read_csv('US_flights_2023.csv')

----
## Limpeza e pré-processamento de dados

Nesta seção, será feita a limpeza da base dados, contemplando somente as informações julgadas relevantes dado o objetivo do projeto.

In [3]:
# Obtendo os dados do aerorpot de Atlanta (ATL)
dados = dados.loc[dados.Dep_Airport == 'ATL', :]

# Verificando os dados
dados.columns, dados.shape, dados.dtypes

(Index(['FlightDate', 'Day_Of_Week', 'Airline', 'Tail_Number', 'Dep_Airport',
        'Dep_CityName', 'DepTime_label', 'Dep_Delay', 'Dep_Delay_Tag',
        'Dep_Delay_Type', 'Arr_Airport', 'Arr_CityName', 'Arr_Delay',
        'Arr_Delay_Type', 'Flight_Duration', 'Distance_type', 'Delay_Carrier',
        'Delay_Weather', 'Delay_NAS', 'Delay_Security', 'Delay_LastAircraft',
        'Manufacturer', 'Model', 'Aicraft_age'],
       dtype='object'),
 (332935, 24),
 FlightDate            object
 Day_Of_Week            int64
 Airline               object
 Tail_Number           object
 Dep_Airport           object
 Dep_CityName          object
 DepTime_label         object
 Dep_Delay              int64
 Dep_Delay_Tag          int64
 Dep_Delay_Type        object
 Arr_Airport           object
 Arr_CityName          object
 Arr_Delay              int64
 Arr_Delay_Type        object
 Flight_Duration        int64
 Distance_type         object
 Delay_Carrier          int64
 Delay_Weather          in

In [4]:
# Descartando colunas que não serão utilizadas
columns_to_drop = ['Tail_Number', 'Dep_CityName', 'Arr_CityName', 'Flight_Duration', 'Manufacturer', 'Model', 'Aicraft_age', 'Dep_Airport']

for column in columns_to_drop:
    dados = dados.drop(column, axis=1)

In [5]:
# Adaptando as colunas para o tipo correto
dados.Day_Of_Week = dados.Day_Of_Week.astype('category')
dados.Dep_Delay_Tag = dados.Dep_Delay_Tag.astype('category')

# Substituindo os valores categóricos por valores numéricos
dados.Dep_Delay_Type = dados.Dep_Delay_Type.astype('category') # Tipo do atraso na partida
dados['Dep_Delay_Type'] = dados['Dep_Delay_Type'].cat.rename_categories({'Low <5min': '0', 'Medium >15min': '1', 'Hight >60min': '2'})

dados.Arr_Delay_Type = dados.Arr_Delay_Type.astype('category') # Tipo do atraso na chegada
dados['Arr_Delay_Type'] = dados['Arr_Delay_Type'].cat.rename_categories({'Low <5min': '0', 'Medium >15min': '1', 'Hight >60min': '2'})

dados.Distance_type = dados.Distance_type.astype('category') # Tipo da distância
dados['Distance_type'] = dados['Distance_type'].cat.rename_categories({'Short Haul >1500Mi': '0', 'Medium Haul <3000Mi': '1', 'Long Haul <6000Mi': '2'})

# Substituindo a data do voo pelo mês
dados['FlightDate'] = pd.to_datetime(dados['FlightDate']).dt.month
dados.rename(columns={'FlightDate': 'Month'}, inplace=True)

# Verificando os dados
display(dados.head(5))
display(dados.columns)
display(dados.shape)

,Month,Day_Of_Week,Airline,DepTime_label,Dep_Delay,Dep_Delay_Tag,Dep_Delay_Type,Arr_Airport,Arr_Delay,Arr_Delay_Type,Distance_type,Delay_Carrier,Delay_Weather,Delay_NAS,Delay_Security,Delay_LastAircraft
31,1,1,Endeavor Air,Afternoon,-3,0,0,FAY,-12,0,0,0,0,0,0,0
32,1,2,Endeavor Air,Afternoon,-5,0,0,FAY,-12,0,0,0,0,0,0,0
33,1,3,Endeavor Air,Afternoon,-2,0,0,FAY,-7,0,0,0,0,0,0,0
34,1,4,Endeavor Air,Afternoon,16,1,1,FAY,11,0,0,0,0,0,0,0
35,1,5,Endeavor Air,Afternoon,-3,0,0,FAY,-8,0,0,0,0,0,0,0


Index(['Month', 'Day_Of_Week', 'Airline', 'DepTime_label', 'Dep_Delay',
       'Dep_Delay_Tag', 'Dep_Delay_Type', 'Arr_Airport', 'Arr_Delay',
       'Arr_Delay_Type', 'Distance_type', 'Delay_Carrier', 'Delay_Weather',
       'Delay_NAS', 'Delay_Security', 'Delay_LastAircraft'],
      dtype='object')

(332935, 16)

----
## Análise exploratória dos dados

In [6]:
dados.dtypes

Month                    int32
Day_Of_Week           category
Airline                 object
DepTime_label           object
Dep_Delay                int64
Dep_Delay_Tag         category
Dep_Delay_Type        category
Arr_Airport             object
Arr_Delay                int64
Arr_Delay_Type        category
Distance_type         category
Delay_Carrier            int64
Delay_Weather            int64
Delay_NAS                int64
Delay_Security           int64
Delay_LastAircraft       int64
dtype: object

In [7]:
contagem = {}
for column in dados.columns:
    if dados[column].dtype.name != 'int64':
        contagem[column] = dados[column].value_counts(True)

In [8]:
dep_delay = dados['Dep_Delay']
arr_delay = dados['Arr_Delay']
total_delay = dep_delay + arr_delay


delay_carrier = dados['Delay_Carrier']
delay_weather = dados['Delay_Weather']
delay_nas = dados['Delay_NAS']
delay_security = dados['Delay_Security']
delay_late_aircraft = dados['Delay_LastAircraft']

total_delay.corr(delay_weather)

0.31595476650356746

----
## Bibliografia

- Fonte do dataset: https://www.kaggle.com/datasets/bordanova/2023-us-civil-flights-delay-meteo-and-aircraft (dados fornecidos pelo Bureau of Transportation Statistics)